In [ ]:
# Verificar que el dataset se puede cargar correctamente
from ultralytics.data.utils import check_det_dataset
import yaml

# Cargar y verificar el archivo de configuración
with open("larc-2/data_fixed.yaml", "r") as f:
    data_config = yaml.safe_load(f)
    
print("Configuración del dataset:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Verificar que el dataset se puede cargar
try:
    dataset_info = check_det_dataset("larc-2/data_fixed.yaml")
    print("\n✅ Dataset verificado exitosamente!")
    print(f"Imágenes de entrenamiento: {len(dataset_info.get('train', []))}")
except Exception as e:
    print(f"\n❌ Error verificando dataset: {e}")

In [ ]:
import torch
import cv2
from roboflow import Roboflow
import dotenv
import os
dotenv.load_dotenv()
print("GPU disponible:", torch.cuda.is_available())

!nvidia-smi

In [ ]:
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace(os.getenv("ROBOFLOW_WORKSPACE")).project(os.getenv("ROBOFLOW_PROJECT_ID"))
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8m model
model = YOLO("yolov8m.pt")

# Train the model with the downloaded dataset
results = model.train(
    data="larc-2/data_fixed.yaml",  # Usar el archivo data.yaml corregido
    epochs=50,
    imgsz=640,
    batch=4,
    split=0.8,  # Dividir automáticamente: 80% entrenamiento, 20% validación
    perspective=0.0005,
    scale=0.4,
    translate=0.04,
    degrees=5,
    shear=1,
    hsv_s=0.5,
    hsv_v=0.3,
    flipud=0.2,
    patience=15,
    save_period=20,
    name="larc-model",
    project="runs/detect",
)

# Validate the model trained
model.val()

In [13]:
# Cargar el modelo entrenado (no el modelo base)
model = YOLO("runs/detect/larc-model/weights/best.pt")

# Probar el modelo con una imagen del dataset
image_path = "larc-2/train/images/IMG_1925_jpg.rf.801203b3907a7cca6b145ead8c948ea1.jpg"
results = model(image_path)

# Mostrar los resultados
annotated_img = results[0].plot()

# Mostrar la imagen con detecciones
cv2.imshow("YOLO inference - Modelo Entrenado", annotated_img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Imprimir información de las detecciones
print(f"Detecciones encontradas: {len(results[0].boxes)}")
if len(results[0].boxes) > 0:
    for i, box in enumerate(results[0].boxes):
        class_id = int(box.cls)
        confidence = float(box.conf)
        class_name = results[0].names[class_id]
        print(f"Detección {i+1}: {class_name} (confianza: {confidence:.2f})")


image 1/1 /home/hector/Desktop/LARC_OPEN2025/vision/model/larc-2/train/images/IMG_1925_jpg.rf.801203b3907a7cca6b145ead8c948ea1.jpg: 640x480 1 mature, 12.3ms
Speed: 1.7ms preprocess, 12.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)
Detecciones encontradas: 1
Detección 1: mature (confianza: 0.97)


In [ ]:
model = YOLO("runs/detect/larc-model/weights/best.pt")

image_path = "larc-2/train/images/IMG_1925_jpg.rf.801203b3907a7cca6b145ead8c948ea1.jpg"